# KBO 1982~2020 투수 기록 크롤링(statiz.co.kr)

## Chrome driver 설치

보편적으로 사용하는 Chrome 웹 브라우저를 이용한 크롤링입니다.

본인이 사용중인 크롬 버전과 맞는 driver를 설치해야 소스코드를 실행할 수 있습니다.

먼저 아래 링크에서 현재 사용중인 크롬 버전을 확인합니다.

[크롬 버전 확인](chrome://version)

가장 첫 줄에 적혀있는 게 크롬 버전입니다.

아래 링크에서 버전과 맞는 드라이버를 설치하면 됩니다.

(크롬을 이참에 업데이트 해주는 것도 좋겠습니다.)

[크롬 드라이버 다운로드](https://sites.google.com/a/chromium.org/chromedriver/downloads)

다운로드하면 압축파일이 하나 있는데, 압축을 해제한 Chrome driver 파일을

이 파일의 경로에 놓으면 소스코드를 그대로 실행하셔도 무방합니다.

In [1]:
# 필요한 라이브러리 import
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time

# setup Driver | Chrome: 크롬 드라이버를 사용하는 driver 생성
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(10)

result = []
rows = 300  # 한 번에 가져올 row 수
reps = 1  # 반복
for i in range(1982, 2021):  # 시작년도, 종료년도 + 1
    count = 0
    for j in range(reps):  
        # url 접근
        driver.get('http://www.statiz.co.kr/stat.php?mid=stat&re=1&ys=' + str(i) + '&ye=' + str(i) + '&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=' + str(rows) + '&pa=' + str(j * rows) + '&si=&cn=')
        
        # 데이터 완전히 읽도록 대기 (데이터 유실되면 sleep을 늘려주거나, rows를 줄이고 reps를 늘려주세요.)
        # 처리 속도를 늘리려면 Redirection을 최소화해야 합니다. rows를 늘리고 reps를 줄인 뒤 sleep을 늘리는게 가장 효율적입니다.
        time.sleep(2)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        tr = soup.select('table#mytable.table.table-striped.table-responsive.table-condensed.no-space.table-bordered > tbody > tr')
        
        # 선수 데이터 row 수가 0인 경우 조기 종료
        if(len(tr) == 0):
            break
            
        # Row 수 계산
        for r in tr:
            row = []
            td = r.find_all('td')
            for d in td:
                if d == td[2]:  # 팀(시즌 + 팀)
                    span = d.find_all('span')
                    year, team, teamTag = span[1].text.strip(), '', ''

                    if int(year) > 81: year = '19' + year
                    else: year = '20' + year

                    teamTag = span[2:]

                    for t in teamTag:
                        team += t.text.strip()

                    row.append(year)  # 시즌
                    row.append(team)  # 팀
                else:
                    row.append(d.text.strip())
            if(len(row) > 0):
                count += 1  # Row 수 세기
                result.append(row)
    # Row 수 출력
    print(i, '년: ', count, sep='')

data = pd.DataFrame(result)

1982년: 43
1983년: 55
1984년: 59
1985년: 69
1986년: 89
1987년: 88
1988년: 92
1989년: 119
1990년: 117
1991년: 136
1992년: 133
1993년: 124
1994년: 141
1995년: 140
1996년: 144
1997년: 144
1998년: 135
1999년: 151
2000년: 154
2001년: 175
2002년: 172
2003년: 171
2004년: 178
2005년: 175
2006년: 172
2007년: 172
2008년: 187
2009년: 192
2010년: 191
2011년: 186
2012년: 188
2013년: 194
2014년: 211
2015년: 244
2016년: 250
2017년: 239
2018년: 260
2019년: 257
2020년: 192


## 불필요한 데이터 제거 및 csv 저장

In [2]:
# 컬럼 확인
data.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,1,박철순,1982,O,9.54,36,15,2,19,24,...,108,0,4,1.84,2.82,0.97,211.2,137.8,9.54,
1,2,황규봉,1982,삼,8.62,47,8,2,16,15,...,99,1,4,2.47,2.67,1.00,160.6,148.5,8.62,
2,3,노상수,1982,롯,7.05,44,12,2,20,14,...,141,2,4,2.94,3.03,1.18,134.9,131.3,7.05,
3,4,하기룡,1982,M,6.78,43,7,4,16,13,...,107,2,6,2.30,2.95,1.11,172.3,134.4,6.78,
4,5,권영호,1982,삼,5.12,32,6,3,21,15,...,72,1,1,2.37,3.91,1.07,167.5,101.3,5.12,


In [2]:
# 중복된 column(WAR) 제거
data = data.drop(data.columns[4], axis='columns')

In [3]:
# 한글 column
data.columns = ['순서', '이름', '시즌', '팀', '출장', '완투', '완봉', '선발', '승리', '패전', '세이브', '홀드', '이닝', '실점', '자책', '상대 타자', '안타', '2루타', '3루타', '피홈런', '볼넷', '고의사구', '사구', '탈삼진', '보크', '폭투', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+', 'WAR', 'WPA']
data.to_csv('KBO_1982_2020_Pitcher_KR.csv', encoding='cp949', index=False)
# 영어 column
data.columns = ['No', 'Name', 'Year', 'Team', 'G', 'CG', 'ShO', 'GS', 'W', 'L', 'SV', 'HLD(HD)', 'IP', 'R', 'ER', 'TBF', 'H', '2B', '3B', 'HR', 'BB', 'IBB', 'HBP', 'SO', 'BK', 'WP', 'ERA', 'FIP', 'WHIP', 'ERA+', 'FIP+', 'WAR', 'WPA']
data.to_csv('KBO_1982_2020_Pitcher_EN.csv', encoding='cp949', index=False)

1
